## 필요한 라이브러리 설치 및 임포트

In [1]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=c6b0cda7813228bc04c8fff47a11bf30ca379cb68a6e25ce43b2860203bd1775
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfram

In [2]:
!pip install konlpy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.6/496.6 kB 34.2 MB/s eta 0:00:00


In [11]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import joblib

## 데이터 전처리하기

In [4]:
# 1. TSV 파일 불러오기
df = pd.read_csv('kr3.tsv', sep='\t')

# 2. CSV 파일로 저장하기
df.to_csv('/content/kr3.csv', index=False)

In [5]:
df = pd.read_csv('/content/kr3.csv')
df.head()

,Rating,Review
0,1,숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편...
1,1,고기가 정말 맛있었어요! 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절...
2,1,"잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어..."
3,1,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
4,1,순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요. 그러나 밥 양...


In [6]:
df.isna().sum()

,0
Rating,0
Review,0


In [7]:
df['Rating'].value_counts(normalize=True)

# 0은 부정, 1은 긍정, 2는 모호함.

,proportion
Rating,
1,0.604758
2,0.284749
0,0.110493


In [8]:
def text_cleaning(text):
  text = re.sub(r"<.*?>|&[^;]+;", " ", text)
  text = re.sub(r"\s+", " ", text)
  text = re.sub(r"[^가-힣a-zA-Z0-9 ]", "", text)

  return text

df['Review'] = df['Review'].apply(lambda x: text_cleaning(x))
df['Review']

,Review
0,숙성 돼지고기 전문점입니다 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편이...
1,고기가 정말 맛있었어요 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절하...
2,잡내 없고 깔끔 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어갈...
3,고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
4,순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요 그러나 밥 양이...
...,...
641757,요즘 핫하게 떠오르고 있는 중국집 맥주의 여 파루 속이 안 좋지만 와봄 일명 선풍...
641758,원래 글 안 쓰는데 이거는 정말 다른 분들 위해서 써야 할 것 같네요 방금 포장 주...
641759,우리 팀 단골집 술 먹고 다음 날 가면 푸짐하게 배불리 해장할 수 있는 곳 주말도 ...
641760,원래는 평택에 있었는데 연남동에도 최근에 생겨서 방문했는데 진짜 줄이 어마어마하더라...


In [9]:
def get_pos(x):
  okt = Okt()
  pos = okt.pos(x)
  pos = ['{}/{}'.format(word, tag) for word, tag in pos if tag not in ['Josa', 'Eomi', 'Punctuation']]
  return pos

result = get_pos(df['Review'][0])
print(result)

['숙성/Noun', '돼지고기/Noun', '전문점/Noun', '입니다/Adjective', '건물/Noun', '모양/Noun', '때문/Noun', '매장/Noun', '모양/Noun', '좀/Noun', '특이하지만/Adjective', '쾌적한/Adjective', '편이/Noun', '살짝/Noun', '레트로/Noun', '감성/Noun', '분위기/Noun', '잡아놨습니다/Verb', '모든/Noun', '직원/Noun', '분들/Suffix', '전부/Noun', '가능하다고/Adjective', '멘트/Noun', '쳐주시며/Verb', '고기/Noun', '초반/Noun', '커팅/Noun', '구워주십니다/Verb', '가격/Noun', '저렴한/Adjective', '편/Noun', '아니지만/Adjective', '맛/Noun', '준수/Noun', '합니다/Verb', '등심/Noun', '덧/Noun', '살이/Noun', '인상/Noun', '깊었는데/Adjective', '구이/Noun', '별로/Noun', '일/Noun', '줄/Noun', '알았는데/Verb', '육/Modifier', '향/Noun', '짙고/Adjective', '얇게/Adjective', '저/Noun', '뻑뻑/Noun', '하지/Verb', '않았습니다/Verb', '하이라이트/Noun', '된장찌개/Noun', '진짜/Noun', '굿/Noun', '입니다/Adjective', '버터/Noun', '간장/Noun', '밥/Noun', '골뱅이/Noun', '국수/Noun', '등/Noun', '나중/Noun', '더/Noun', '맛봐야/Verb', '할/Verb', '것/Noun', '들/Suffix', '남겨/Verb', '뒀습니다/Verb']


In [13]:
tqdm.pandas()

# 형태소 분석 진행상황 보기
df['pos'] = df['Review'].progress_apply(get_pos)

# 이미 토큰화된 데이터를 그대로 사용하기 위한 설정
def identity_tokenizer(x):
    return x

def identity_preprocessor(x):
    return x

# 벡터라이저 생성
tfidvect = TfidfVectorizer(
    tokenizer=identity_tokenizer,
    preprocessor=identity_preprocessor,
    token_pattern=None
)
X = tfidvect.fit_transform(df['pos'])

# 라벨 추출
y = df['Rating']

# 학습/테스트 분할
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("X_train shape:", x_train.shape)
print("X_test shape:", x_test.shape)

100%|██████████| 641762/641762 [3:26:50<00:00, 51.71it/s]


X_train shape: (513409, 247724)
X_test shape: (128353, 247724)


In [14]:
lr = LogisticRegression(random_state=0, class_weight='balanced', max_iter=1000)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.84      0.72     14182
           1       0.84      0.73      0.78     77623
           2       0.53      0.61      0.57     36548

    accuracy                           0.71    128353
   macro avg       0.67      0.73      0.69    128353
weighted avg       0.73      0.71      0.71    128353



In [15]:
# 모델과 벡터 저장
joblib.dump(lr, 'sentiment_model.pkl')
joblib.dump(tfidvect, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']